### Set up

In [6]:
!pip install openpyxl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.0/250.0 kB 1.7 MB/s eta 0:00:000:00:010:00:01:01


In [1]:
import geopandas as gpd
import pandas as pd
import openpyxl
from datetime import datetime

In [2]:
path_in = "/Users/sofia/Documents/Repos/skytruth-30x30/data/data/raw/"
path_out = "/Users/sofia/Documents/Repos/skytruth-30x30/data/data/processed/"

### Process habitats from [Ocean+](https://habitats.oceanplus.org/) (except mangroves)

In [3]:
cold = pd.read_csv(path_in + "Ocean+HabitatsDownload_Global/coldwatercorals.csv")
salt = pd.read_csv(path_in + "Ocean+HabitatsDownload_Global/saltmarshes.csv")
sea = pd.read_csv(path_in + "Ocean+HabitatsDownload_Global/seagrasses.csv")
warm = pd.read_csv(path_in + "Ocean+HabitatsDownload_Global/warmwatercorals.csv")
glob = pd.read_excel(path_in + "Ocean+HabitatsDownload_Global/global-stats.xlsx")

In [4]:
cold2 = cold[['ISO3', 'protected_area', 'total_area']].rename(columns={'ISO3': 'location_id'})
salt2 = salt[['ISO3', 'protected_area', 'total_area']].rename(columns={'ISO3': 'location_id'})
sea2 = sea[['ISO3', 'protected_area', 'total_area']].rename(columns={'ISO3': 'location_id'})
warm2 = warm[['ISO3', 'protected_area', 'total_area']].rename(columns={'ISO3': 'location_id'})

In [5]:
# Remove rows with '-' in 'protected_area' or 'total_area'
cold2 = cold2[~cold2['protected_area'].str.contains('-') & ~cold2['total_area'].str.contains('-')]
salt2 = salt2[~salt2['protected_area'].str.contains('-') & ~salt2['total_area'].str.contains('-')]
sea2 = sea2[~sea2['protected_area'].str.contains('-') & ~sea2['total_area'].str.contains('-')]
warm2 = warm2[~warm2['protected_area'].str.contains('-') & ~warm2['total_area'].str.contains('-')]


In [6]:
# Bring the wdpa file to get the iso3 and parent_iso equivalences
wdpa = gpd.read_file(path_out + "wdpa/merged_mpa.shp")

# Filter out rows with multiple values in either 'ISO3' or 'PARENT_ISO'
wdpa = wdpa[~wdpa['ISO3'].str.contains(';') & ~wdpa['PARENT_ISO'].str.contains(';')]

# Extract unique ISO3-PARENT_ISO pairs
unique_pairs = wdpa[['ISO3', 'PARENT_ISO']].drop_duplicates()

In [7]:
# Create a mapping dictionary for ISO3-PARENT_ISO pairs and modify the 'location_id' column in the habitats dataframes
mapping_dict = dict(zip(unique_pairs['ISO3'], unique_pairs['PARENT_ISO']))
cold2['location_id'] = cold2['location_id'].map(mapping_dict)
salt2['location_id'] = salt2['location_id'].map(mapping_dict)
sea2['location_id'] = sea2['location_id'].map(mapping_dict)
warm2['location_id'] = warm2['location_id'].map(mapping_dict)

In [8]:
# Group by 'location_id' and calculate the sum of 'protected_area' and 'total_area'
cold2_grouped = cold2.groupby('location_id').sum().reset_index()
salt2_grouped = salt2.groupby('location_id').sum().reset_index()
sea2_grouped = sea2.groupby('location_id').sum().reset_index()
warm2_grouped = warm2.groupby('location_id').sum().reset_index()

In [9]:
# Add the 'habitat_name' column
cold2_grouped['habitat_name'] = 'cold-water corals'
salt2_grouped['habitat_name'] = 'saltmarshes'
sea2_grouped['habitat_name'] = 'seagrasses'
warm2_grouped['habitat_name'] = 'warm-water corals'

In [10]:
# Concatenate the dataframes
habitats = pd.concat([cold2_grouped, salt2_grouped, sea2_grouped, warm2_grouped])
habitats['year'] = datetime.now().year
habitats.head(2)

,location_id,protected_area,total_area,habitat_name,year
0,ABNJ,421.629372679904,1874.98221422617,cold-water corals,2023
1,AGO,0,3.39567053773998,cold-water corals,2023


In [11]:
# Calculate global stats for habitats
habitats_global = glob[['habitat','protected_area', 'total_area']].rename(columns={'habitat': 'habitat_name'})
habitats_global['location_id'] = 'GLOB'
habitats_global['year'] = datetime.now().year
habitats_global = habitats_global[habitats_global['habitat_name'] != 'mangroves'] # remove mangroves
habitats_global

,habitat_name,protected_area,total_area,location_id,year
0,saltmarsh,111638.252564,224435.075094,GLOB,2023
1,seagrass,74787.449960,314001.940600,GLOB,2023
2,warmwater-corals,63259.499130,149886.974126,GLOB,2023
4,coldwater-corals,4400.140842,15336.975280,GLOB,2023


In [12]:
# Change the name of the habitats to match the ones in the habitats dataframe
habitat_name_mapping = {
    'saltmarsh': 'saltmarshes',
    'seagrass': 'seagrasses',
    'warmwater-corals': 'warm-water corals',
    'coldwater-corals': 'cold-water corals'
}
habitats_global['habitat_name'] = habitats_global['habitat_name'].replace(habitat_name_mapping)
habitats_global

,habitat_name,protected_area,total_area,location_id,year
0,saltmarshes,111638.252564,224435.075094,GLOB,2023
1,seagrasses,74787.449960,314001.940600,GLOB,2023
2,warm-water corals,63259.499130,149886.974126,GLOB,2023
4,cold-water corals,4400.140842,15336.975280,GLOB,2023


In [13]:
# Concatenate the global stats to the habitats dataframe
habitats = pd.concat([habitats, habitats_global])
habitats['habitat_name'].unique()

array(['cold-water corals', 'saltmarshes', 'seagrasses',
       'warm-water corals'], dtype=object)

In [14]:
# List of dictionaries for data in Region_ISO3_PP.txt (list of regions used in the Protected Planet database)
regions_data = [
    {
        'region_iso': 'AS',
        'region_name': 'Asia & Pacific',
        'country_iso_3s': [
            "AFG", "ASM", "AUS", "BGD", "BRN", "BTN", "CCK", "CHN", "COK", "CXR", "FJI", "FSM", "GUM", "HKG", "IDN",
            "IND", "IOT", "IRN", "JPN", "KHM", "KIR", "KOR", "LAO", "LKA", "MAC", "MDV", "MHL", "MMR", "MNG", "MNP",
            "MYS", "NCL", "NFK", "NIU", "NPL", "NRU", "NZL", "PAK", "PCN", "PHL", "PLW", "PNG", "PRK", "PYF", "SGP",
            "SLB", "THA", "TKL", "TLS", "TON", "TUV", "TWN", "VNM", "VUT", "WLF", "WSM"
        ]
    },
    {
        'region_iso': 'AF',
        'region_name': 'Africa',
        'country_iso_3s': [
            "AGO", "BDI", "BEN", "BFA", "BWA", "CAF", "CIV", "CMR", "COD", "COG", "COM", "CPV", "DJI", "DZA", "EGY",
            "ERI", "ESH", "ETH", "GAB", "GHA", "GIN", "GMB", "GNB", "GNQ", "KEN", "LBR", "LBY", "LSO", "MAR", "MDG",
            "MLI", "MOZ", "MRT", "MUS", "MWI", "MYT", "NAM", "NER", "NGA", "REU", "RWA", "SDN", "SEN", "SHN", "SLE",
            "SOM", "SSD", "STP", "SWZ", "SYC", "TCD", "TGO", "TUN", "TZA", "UGA", "ZAF", "ZMB", "ZWE"
        ]
    },
    {
        'region_iso': 'EU',
        'region_name': 'Europe',
        'country_iso_3s': [
            "ALA", "ALB", "AND", "ARM", "AUT", "AZE", "BEL", "BGR", "BIH", "BLR", "CHE", "CYP", "CZE", "DEU", "DNK",
            "ESP", "EST", "FIN", "FRA", "FRO", "GBR", "GEO", "GGY", "GIB", "GRC", "HRV", "HUN", "IMN", "IRL", "ISL",
            "ISR", "ITA", "JEY", "KAZ", "KGZ", "LIE", "LTU", "LUX", "LVA", "MCO", "MDA", "MKD", "MLT", "MNE", "NLD",
            "NOR", "POL", "PRT", "ROU", "RUS", "SJM", "SMR", "SRB", "SVK", "SVN", "SWE", "TJK", "TKM", "TUR", "UKR",
            "UZB", "VAT"
        ]
    },
    {
        'region_iso': 'SA',
        'region_name': 'Latin America & Caribbean',
        'country_iso_3s': [
            "ABW", "AIA", "ARG", "ATG", "BES", "BHS", "BLM", "BLZ", "BMU", "BOL", "BRA", "BRB", "CHL", "COL", "CRI",
            "CUB", "CUW", "CYM", "DMA", "DOM", "ECU", "FLK", "GLP", "GRD", "GTM", "GUF", "GUY", "HND", "HTI", "JAM",
            "KNA", "LCA", "MAF", "MEX", "MSR", "MTQ", "NIC", "PAN", "PER", "PRI", "PRY", "SLV", "SUR", "SXM", "TCA",
            "TTO", "UMI", "URY", "VCT", "VEN", "VGB", "VIR"
        ]
    },
    {
        'region_iso': 'PO',
        'region_name': 'Polar',
        'country_iso_3s': [
            "ATF", "BVT", "GRL", "HMD", "SGS"
        ]
    },
    {
        'region_iso': 'NA',
        'region_name': 'North America',
        'country_iso_3s': [
            "CAN", "SPM", "USA"
        ]
    },
    
    {
        'region_iso': 'WA',
        'region_name': 'West Asia',
        'country_iso_3s': [
            "ARE", "BHR", "IRQ", "JOR", "KWT", "LBN", "OMN", "PSE", "QAT", "SAU", "SYR", "YEM"
        ]
    },
    {
        'region_iso': 'AT', # this region is not in the Protected Planet database
        'region_name': 'Antartica',
        'country_iso_3s': [
            "ATA"
        ]
    }
]

# Convert the region data to a dictionary that maps each country to its region name
country_to_region = {}
for region in regions_data:
    for country in region['country_iso_3s']:
        country_to_region[country] = region['region_iso']

In [15]:
# Add regions field
habitats_regions = habitats.copy()
habitats_regions['region'] = habitats['location_id'].map(country_to_region)

# Convert fields to numeric
habitats_regions['protected_area'] = pd.to_numeric(habitats_regions['protected_area'], errors='coerce')
habitats_regions['total_area'] = pd.to_numeric(habitats_regions['total_area'], errors='coerce')

# Calculate stats for each region
habitats_regions = habitats_regions.groupby(['region', 'habitat_name']).agg({'protected_area': 'sum', 'total_area': 'sum'}).reset_index()
habitats_regions['year'] = datetime.now().year
habitats_regions.rename(columns={'region': 'location_id'}, inplace=True)
habitats_regions


,location_id,habitat_name,protected_area,total_area,year
0,AF,cold-water corals,37.761626,381.993234,2023
1,AF,saltmarshes,6688.702879,19845.915000,2023
2,AF,seagrasses,6319.099491,61939.484904,2023
3,AF,warm-water corals,6591.340083,15216.393947,2023
4,AS,cold-water corals,263.251498,1332.225080,2023
5,AS,saltmarshes,11721.439539,39229.888860,2023
6,AS,seagrasses,28942.705660,72666.482052,2023
7,AS,warm-water corals,13895.870659,67363.486609,2023
8,EU,cold-water corals,2183.050266,6179.526427,2023
9,EU,saltmarshes,7431.043710,13274.326478,2023


In [16]:
# Concatenate region statistics to the habitats dataframe
habitats = pd.concat([habitats, habitats_regions])

In [32]:
habitats['location_id'].unique()

array(['ABNJ', 'AGO', 'ALB', 'ARG', 'ATG', 'AUS', 'BHS', 'BLZ', 'BRA',
       'BRB', 'CAN', 'CHL', 'CHN', 'CIV', 'COK', 'COL', 'CPV', 'CRI',
       'CUB', 'CYP', 'DMA', 'DNK', 'DOM', 'DZA', 'ECU', 'ERI', 'ESP',
       'FJI', 'FRA', 'FSM', 'GBR', 'GHA', 'GIN', 'GNB', 'GNQ', 'GRC',
       'GRD', 'GTM', 'GUY', 'HND', 'HRV', 'HTI', 'IDN', 'IND', 'IRL',
       'ISL', 'ITA', 'JAM', 'JPN', 'KEN', 'KIR', 'KNA', 'LBR', 'LCA',
       'LKA', 'MAR', 'MDG', 'MEX', 'MHL', 'MLT', 'MMR', 'MNE', 'MOZ',
       'MRT', 'MUS', 'MYS', 'NAM', 'NGA', 'NIC', 'NLD', 'NOR', 'NZL',
       'OMN', 'PAN', 'PER', 'PHL', 'PLW', 'PNG', 'PRT', 'RUS', 'SAU',
       'SEN', 'SHN', 'SJM', 'STP', 'SUR', 'SWE', 'SYC', 'THA', 'TLS',
       'TON', 'TTO', 'TUN', 'TUV', 'UMI', 'URY', 'USA', 'VCT', 'VEN',
       'VNM', 'VUT', 'WSM', 'YEM', 'ZAF', 'ARE', 'AZE', 'BEL', 'BGR',
       'BHR', 'DEU', 'DJI', 'EGY', 'EST', 'FIN', 'GEO', 'GMB', 'IRN',
       'KHM', 'KOR', 'KWT', 'LBY', 'LTU', 'LVA', 'PAK', 'POL', 'QAT',
       'ROU', 'SDN'

In [33]:
habitats.to_csv(path_out + "habitats/ocean+.csv", index=False)

### Process seamounts from [UN WCMC](https://data.unep-wcmc.org/datasets/41)

In [52]:
# Read required data
seamounts = gpd.read_file(path_in + "Seamounts/DownloadPack-14_001_ZSL002_ModelledSeamounts2011_v1_01_Data_Seamounts_Seamounts.shp")
eez = gpd.read_file(path_out + "/administrative/eez_area_mollweide.shp")
hs = gpd.read_file(path_in + "/high_seas/high_seas.shp")
protected_areas = gpd.read_file(path_out + "wdpa/timeseries/protected_dissolved_2023.shp").to_crs("EPSG:4326")

In [53]:
# Keep relevant fields in eez and hs and merge then in one dataframe
eez = eez[['SOVEREIGN1', 'SOVEREIGN2', 'SOVEREIGN3','ISO_SOV1', 'ISO_SOV2', 'ISO_SOV3', 'geometry']]
hs = hs[['geometry']]
hs['SOVEREIGN1'] = 'High Seas'
hs['ISO_SOV1'] = 'ABNJ'
eez_hs = eez.merge(hs, how='outer')

In [54]:
# Join eez info to seamounts falling within eez polygons
seamounts_eez = gpd.sjoin(seamounts, eez_hs, how="left", predicate="within")
# Drop those not associated with an eez or hs
seamounts_eez = seamounts_eez.dropna(subset=['ISO_SOV1'])

In [55]:
# Create new column "iso" with the iso_sov codes
def concatenate_iso(row):
    iso_list = [row['ISO_SOV1']]
    if not pd.isna(row['ISO_SOV2']):
        iso_list.append(row['ISO_SOV2'])
    if not pd.isna(row['ISO_SOV3']):
        iso_list.append(row['ISO_SOV3'])
    return ';'.join(iso_list)

seamounts_eez['iso'] = seamounts_eez.apply(concatenate_iso, axis=1)

In [56]:
# Check which seamounts are protectec
seamounts_wdpa = gpd.sjoin(seamounts, protected_areas, how="left", predicate="within")
seamounts_wdpa['protection'] = "no"  
seamounts_wdpa.loc[~seamounts_wdpa['index_right'].isna(), 'protection'] = "yes"
# Remove rows in which protection is "no"
seamounts_wdpa = seamounts_wdpa[seamounts_wdpa['protection'] != "no"]

### Global stats

In [65]:
# Calculate global area of seamounts
seamounts_eez['habitat_name'] = 'seamounts'
seamounts_global = seamounts_eez.groupby(['habitat_name']).agg({'AREA2D': 'sum'}).reset_index().rename(columns={'AREA2D': 'total_area'})
seamounts_global['location_id'] = 'GLOB'
seamounts_global['year'] = 2011
seamounts_global

,habitat_name,total_area,location_id,year
0,seamounts,2.690810e+07,GLOB,2011


In [66]:
# Calculate global area of seamounts protected
seamounts_wdpa['habitat_name'] = 'seamounts'
seamounts_wdpa_global = seamounts_wdpa.groupby(['habitat_name']).agg({'AREA2D': 'sum'}).reset_index().rename(columns={'AREA2D': 'protected_area'})
seamounts_wdpa_global

,habitat_name,protected_area
0,seamounts,3.438552e+06


In [67]:
# Bring 'protected_area' field from seamouts_wdpa_global to seamounts_global
seamounts_global = seamounts_global.merge(seamounts_wdpa_global[['habitat_name', 'protected_area']], how='left', on='habitat_name')
seamounts_global

,habitat_name,total_area,location_id,year,protected_area
0,seamounts,2.690810e+07,GLOB,2011,3.438552e+06


### Country stats

In [68]:
# Split the 'iso_code' values and create separate rows only for those with multiple values
mask = seamounts_eez['iso'].str.contains(';', na=False)
split_rows = seamounts_eez[mask].copy()
split_rows['iso'] = split_rows['iso'].str.split(';')
split_rows = split_rows.explode('iso')

# Keep rows with single values in 'iso_code'
single_value_rows = seamounts_eez[~mask]

# Concatenate the exploded rows with the single value rows
seamounts_eez_new = pd.concat([single_value_rows, split_rows], ignore_index=True)

In [69]:
# Get area of seamounts per iso
seamounts_iso = seamounts_eez_new.groupby(['iso']).agg({'AREA2D': 'sum'}).reset_index()
seamounts_iso = seamounts_iso.rename(columns={'AREA2D': 'total_area', 'iso': 'location_id'})
seamounts_iso['habitat_name'] = 'seamounts'
seamounts_iso['year'] = 2011
seamounts_iso 

,location_id,total_area,habitat_name,year
0,ABNJ,1.483098e+07,seamounts,2011
1,AGO,9.556242e+03,seamounts,2011
2,ARG,3.110730e+05,seamounts,2011
3,ATA,3.551629e+05,seamounts,2011
4,ATG,6.215895e+03,seamounts,2011
...,...,...,...,...
88,VNM,4.421338e+04,seamounts,2011
89,VUT,1.199475e+05,seamounts,2011
90,WSM,4.117997e+04,seamounts,2011
91,YEM,6.294974e+04,seamounts,2011


In [70]:
# Split the 'iso_code' values in seamounts_wdpa and create separate rows only for those with multiple values
mask = seamounts_wdpa['PARENT_ISO'].str.contains(';', na=False)
split_rows = seamounts_wdpa[mask].copy()
split_rows['PARENT_ISO'] = split_rows['PARENT_ISO'].str.split(';')
split_rows = split_rows.explode('PARENT_ISO')

# Keep rows with single values in 'iso_code'
single_value_rows = seamounts_wdpa[~mask]

# Concatenate the exploded rows with the single value rows
seamounts_wdpa_new = pd.concat([single_value_rows, split_rows], ignore_index=True)

In [71]:
# Calculate area protected per iso
seamounts_protected = seamounts_wdpa_new.groupby(['PARENT_ISO']).agg({'AREA2D': 'sum'}).reset_index()
seamounts_protected = seamounts_protected.rename(columns={'AREA2D': 'protected_area', 'PARENT_ISO': 'location_id'})
seamounts_protected.head(2)

,location_id,protected_area
0,ABNJ,226253.932283
1,ARG,38773.659962


In [73]:
# Join seamounts_iso and seamounts_protected
seamounts_iso = seamounts_iso.merge(seamounts_protected, how='left', on='location_id')
seamounts_iso.head(2)

,location_id,total_area,habitat_name,year,protected_area
0,ABNJ,1.483098e+07,seamounts,2011,226253.932283
1,AGO,9.556242e+03,seamounts,2011,NaN


### Regions stats

In [74]:
seamounts_regions = seamounts_iso.copy()
seamounts_regions['region'] = seamounts_regions['location_id'].map(country_to_region)

# Calculate stats for each region
seamounts_regions = seamounts_regions.groupby(['region', 'habitat_name']).agg({'protected_area': 'sum', 'total_area': 'sum'}).reset_index()
seamounts_regions['year'] = 2011
seamounts_regions.rename(columns={'region': 'location_id'}, inplace=True)
seamounts_regions

,location_id,habitat_name,protected_area,total_area,year
0,AF,seamounts,94385.178958,6.162351e+05,2011
1,AS,seamounts,832497.783937,5.433433e+06,2011
2,AT,seamounts,0.000000,3.551629e+05,2011
3,EU,seamounts,894514.910255,2.641119e+06,2011
4,NA,seamounts,555588.210725,1.664794e+06,2011
5,SA,seamounts,581172.154389,1.655552e+06,2011
6,WA,seamounts,2487.428050,9.384765e+04,2011


In [75]:
# Concatenate region and global stats to seamounts_iso2
seamounts_all = pd.concat([seamounts_iso, seamounts_regions, seamounts_global])

In [77]:
seamounts_all.to_csv(path_out + "habitats/seamounts.csv", index=False)

### Process mangroves from GMW

In [79]:
mangroves = pd.read_csv(path_out + "habitats/mangroves.csv")

In [80]:
# Calculate global stats for mangroves
mangroves_global = mangroves.groupby(['habitat_name']).agg({'protected_area': 'sum', 'total_area': 'sum'}).reset_index()
mangroves_global['location_id'] = 'GLOB'
mangroves_global['year'] = 2020
mangroves_global

,habitat_name,protected_area,total_area,location_id,year
0,mangroves,61287.20375,147358.990971,GLOB,2020


In [81]:
# Concatenate the global stats to the mangroves dataframe
mangroves = pd.concat([mangroves, mangroves_global])

In [82]:
mangroves_regions = mangroves.copy()
mangroves_regions['region'] = mangroves['location_id'].map(country_to_region)

# Calculate stats for each region
mangroves_regions = mangroves_regions.groupby(['region', 'habitat_name']).agg({'protected_area': 'sum', 'total_area': 'sum'}).reset_index()
mangroves_regions['year'] = 2020
mangroves_regions.rename(columns={'region': 'location_id'}, inplace=True)
mangroves_regions

,location_id,habitat_name,protected_area,total_area,year
0,AF,mangroves,10006.97000,29344.404399,2020
1,AS,mangroves,21378.75000,74629.194446,2020
2,NA,mangroves,2055.40000,2329.115505,2020
3,PO,mangroves,6.72000,6.723018,2020
4,SA,mangroves,27811.53375,40875.932666,2020
5,WA,mangroves,27.83000,173.620938,2020


In [83]:
# Concatenate stats for regions with mangroves
mangroves = pd.concat([mangroves, mangroves_regions])

### Concatenate all habitats

In [85]:
# Concatenate the dataframes
habitats_all = pd.concat([habitats, seamounts_all, mangroves])
habitats_all

,location_id,protected_area,total_area,habitat_name,year
0,ABNJ,421.629372679904,1874.98221422617,cold-water corals,2023
1,AGO,0,3.39567053773998,cold-water corals,2023
2,ALB,0,5.98647948252716,cold-water corals,2023
3,ARG,6.98422602063557,61.8263440651753,cold-water corals,2023
4,ATG,0,0.997746538545076,cold-water corals,2023
...,...,...,...,...,...
1,AS,21378.75,74629.194446,mangroves,2020
2,NA,2055.4,2329.115505,mangroves,2020
3,PO,6.72,6.723018,mangroves,2020
4,SA,27811.53375,40875.932666,mangroves,2020


In [86]:
habitats_all.to_csv(path_out + "tables/habitats2.csv", index=False)